In [ ]:
import os

import tensorflow as tf
import numpy as np
from pandas import DataFrame as DF

import mcbn.data.dataset_loaders as dl
from mcbn.data.dataset import Dataset

from mcbn.models.model_bn import ModelBN
from mcbn.models.model_do import ModelDO

from mcbn.utils.metrics import rmse
from mcbn.utils.helper import get_setup
from mcbn.utils.helper import get_lambdas_range
from mcbn.utils.helper import get_train_and_evaluation_models
from mcbn.utils.helper import get_new_dir_in_parent_path
from mcbn.utils.helper import make_path_if_missing
from mcbn.utils.helper import dump_yaml
from mcbn.utils.helper import get_logger

from mcbn.environment.constants import HYPERPARAMS_EVAL_PATH

In [ ]:
logger = get_logger()

logger.info("STEP 2: Get best hyperparameters")

RANDOM_SEED_NP = 1
RANDOM_SEED_TF = 1

s = get_setup()

# Get lambdas range
s['lambdas'] = get_lambdas_range(s['lambda_min'], s['lambda_max'])

# Tau must be set, but has no relevance for this optimization
TAU = 1

In [ ]:
def get_cv_rmses(c, test_path, folds_path, X, y):
    
    # Store a graph for each fold in a list
    models = []
    datasets = []

    # Get and initialize model
    with tf.Graph().as_default() as g:

        with g.device('/cpu:0'):
            
            # Set random generator seed for reproducible models
            tf.set_random_seed(RANDOM_SEED_TF)
            
            for fold in range(s['n_folds']):
                
                # Get dataset for fold
                X_train, y_train, X_val, y_val = dl.load_fold(folds_path, fold, X, y)
                dataset = Dataset(X_train, 
                                  y_train, 
                                  X_val, 
                                  y_val, 
                                  s['discard_leftovers'],
                                  normalize_X=s['normalize_X'], 
                                  normalize_y=s['normalize_y'])

                # Store dataset in datasets list
                datasets.append(dataset)
                
                with tf.name_scope("model_{}".format(fold)) as scope:
                    
                    # Get graph
                    if 'BN' == c['base_model_name']:
                        model = ModelBN(s['n_hidden'],
                                        K=c['k'],
                                        nonlinearity=s['nonlinearity'],
                                        bn=True,
                                        do=False,
                                        tau=TAU,
                                        dataset=dataset,
                                        in_dim=c['in_dim'],
                                        out_dim=c['out_dim'])
                    elif 'DO' == c['base_model_name']:
                        keep_prob = 1 - c['dropout']
                        model = ModelDO(s['n_hidden'], 
                                        K=c['k'], 
                                        nonlinearity=s['nonlinearity'], 
                                        bn=False, 
                                        do=True,
                                        tau=TAU, 
                                        dataset=dataset, 
                                        in_dim=c['in_dim'], 
                                        out_dim=c['out_dim'],
                                        first_layer_do=True)
                    model.initialize(l2_lambda=c['lambda'], learning_rate=s['learning_rate'])

                    # Store graph in models list
                    models.append(model)
        
        # Create savers to save models after training - one per fold
        savers = [tf.train.Saver(
                        var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='model_{}'.format(fold))
                  ) for fold in range(s['n_folds'])]
        
        # Start session (regular session is default session in with statement)
        with tf.Session(config=tf.ConfigProto(
                allow_soft_placement=False,
                log_device_placement=False,
                inter_op_parallelism_threads=1,
                intra_op_parallelism_threads=1)) as sess:

            sess.run(tf.global_variables_initializer())

            # Keep track of best average RMSE over all folds
            best_results = {em: {'epoch': [0 for n in range(s['n_folds'])], 
                                 'RMSEs': [np.inf for n in range(s['n_folds'])]}
                            for em in c['evaluation_models']}
            unfavorable_evaluations = {em: 0 for em in c['evaluation_models']}
            eval_interval = s['hyperparam_eval_interval']
            
            while any(unfavorable_evaluations[em] < s['patience'] for em in c['evaluation_models']):
                
                curr_results = {em: [] for em in c['evaluation_models']}
                
                # Iterate over all (base) model graphs and corresponding folds
                for i, (model, dataset) in enumerate(zip(models, datasets)):
                    
                    start_epoch = dataset.curr_epoch
                    
                    # Train model for eval_interval iterations
                    while not dataset.at_end_of_epoch(start_epoch + eval_interval, c['batch_size']):
                        batch = dataset.next_batch(c['batch_size'])
                        if 'BN' == c['base_model_name']:
                            model.run_train_step(batch)
                        elif 'DO' == c['base_model_name']:
                            model.run_train_step(batch, keep_prob)
                    
                    # Check that we haven't exceeded global maximum epochs limit
                    # If so, break while loop immediately
                    if dataset.curr_epoch > s['global_max_epochs']:
                        break

                    # Evaluate fold validation RMSE and append to results
                    for em in [em for em in c['evaluation_models'] if not unfavorable_evaluations[em] >= s['patience']]:
                        if em == 'MCBN':
                            yHat, _ = model.predict_mc(s['mc_samples'], dataset.X_test, c['batch_size'])
                        elif em == 'BN':
                            model.update_layer_statistics(dataset.X_train)
                            yHat = model.predict(dataset.X_test)
                        elif em == 'MCDO':
                            yHat, _ = model.predict_mc(s['mc_samples'], dataset.X_test, keep_prob)
                        elif em == 'DO':
                            yHat = model.predict(dataset.X_test, 1)
                        curr_results[em].append(rmse(yHat, dataset.y_test))

                # RMSE at eval_interval end found for all folds
                # For each evaluation model, check if an average RMSE improvement was made
                for em in [em for em in c['evaluation_models'] if not unfavorable_evaluations[em] >= s['patience']]:
                    
                    if np.mean(curr_results[em]) <= np.mean(best_results[em]['RMSEs']):

                        # Store the new best results
                        best_results[em]['epoch'] = [dataset.curr_epoch for dataset in datasets]
                        best_results[em]['RMSEs'] = curr_results[em]
                        unfavorable_evaluations[em] = 0

                        # Save improved models for all folds
                        for fold in range(s['n_folds']):
                            trained_model_dir = os.path.join(test_path, em, 'fold_{}'.format(fold))
                            make_path_if_missing(trained_model_dir)
                            trained_model_file_path = os.path.join(trained_model_dir, 'model')
                            savers[fold].save(sess, trained_model_file_path)                    
                    else:
                        unfavorable_evaluations[em] += 1

                    logger.info(("{}: epochs: {:.2f}, Curr mean: {:.4f}, " +
                           "best mean: {:.4f} at epoch {:.2f}. Breaks: {}").format(
                        em,
                        np.mean([dataset.curr_epoch for dataset in datasets]),
                        np.mean(curr_results[em]),
                        np.mean(best_results[em]['RMSEs']),
                        np.mean(best_results[em]['epoch']),
                        unfavorable_evaluations[em]
                    ))
    
    # Convert best results to DataFrame and return
    test_results_df = None
    for em, em_results in best_results.iteritems():
        model_df = DF(em_results)
        model_df['model'] = em
        test_results_df = model_df if test_results_df is None else test_results_df.append(model_df, ignore_index=True)
    return test_results_df

In [ ]:
def get_config_results(config, base_model_path, X, y, results_df):
    
    # Set random generator seed for consistent batches
    np.random.seed(RANDOM_SEED_NP)
    
    # Each test is a learning parameter combination
    test_count = len(os.listdir(base_model_path))
    test_path = get_new_dir_in_parent_path(base_model_path, 'test_' + str(test_count))
    
    # Get and save results
    folds_path = os.path.join(base_model_path, '..', 'fold_indices')
    test_results_df = get_cv_rmses(config, test_path, folds_path, X, y)
    file_path = os.path.join(test_path, 'cv_results.csv')
    test_results_df.to_csv(file_path)
    
    # Save config
    dump_yaml(config, test_path, 'config.yml')

    # Save results in collection (dropout can be None)
    test_results_df = test_results_df.groupby('model')['RMSEs', 'epoch'].mean().reset_index()
    test_results_df = test_results_df.rename(columns={'RMSEs': 'cv_rmse', 'epoch': 'cv_epoch'})
    for k in ['dataset_name', 'batch_size', 'lambda', 'dropout']:
        test_results_df[k] = config.get(k)
    
    # Print and return results for this test
    logger.info(test_results_df)
    return test_results_df if results_df is None else results_df.append(test_results_df, ignore_index=True)

# Do grid search

In [ ]:
# Results DataFrame
results_df = None

# Create parent evaluation dir
eval_path = get_new_dir_in_parent_path(HYPERPARAMS_EVAL_PATH)

# Save setup
dump_yaml(s, eval_path, 'eval_setup.yml')

for dataset_name in s['datasets']:
    logger.info("Evaluating dataset " + dataset_name)
    
    # Load dataset in memory
    X, y = dl.load_uci_data_full(dataset_name)
    
    # Create eval dir for dataset
    dataset_path = get_new_dir_in_parent_path(eval_path, dataset_name)
    
    # Set random generator seed for reproducible folds
    np.random.seed(RANDOM_SEED_NP)
    
    # Generate folds for this dataset
    folds_path = dl.create_folds(dataset_name, s['n_folds'], s['inverted_cv_fraction'], dataset_path)

    # Get dataset configuration
    feature_indices, target_indices = dl.load_uci_info(dataset_name)

    # Iterate over base models ('BN' and/or 'DO')
    train_and_evaluation_models = get_train_and_evaluation_models(s['models'])
    for base_model in train_and_evaluation_models.keys():
        logger.info("Evaluating base model " + base_model)
        
        c = {'dataset_name': dataset_name,
         'in_dim': len(feature_indices),
         'out_dim': len(target_indices),
         'k': s['k_specific'].get(dataset_name) or s['k'],
         'base_model_name': base_model,
         'evaluation_models': train_and_evaluation_models[base_model]
        }
        
        # Create an eval dir for the base model
        model_path = get_new_dir_in_parent_path(dataset_path, base_model)
        
        # Start evaluation for this base model
        if base_model == 'BN':
            
            # Iterate over batch sizes
            for bs in s['batch_sizes_specific'].get(dataset_name, s['batch_sizes']):
                c['batch_size'] = bs
            
                # Iterate over L2 regularization lambdas
                for l in s['lambdas']:
                    c['lambda'] = l
                    results_df = get_config_results(c, model_path, X, y, results_df)
                    
        # For MDCO, iterate over dropout probabilities
        elif base_model == 'DO':
            
            # Batch size is fixed
            c['batch_size'] = s['dropout_batch_size']
            
            # Iterate over dropout probabilities
            for dropout in s['dropouts']:
                c['dropout'] = dropout
            
                # Iterate over L2 regularization lambdas
                for l in s['lambdas']:
                    c['lambda'] = l
                    results_df = get_config_results(c, model_path, X, y, results_df)
    
    # Save summary for dataset
    dataset_results_path = os.path.join(dataset_path, 'dataset-results.csv')
    dataset_df = results_df[results_df['dataset_name'] == dataset_name].reset_index(drop = True)
    dataset_df.to_csv(dataset_results_path)

# Save dataframe with all results
all_results_path = os.path.join(eval_path, 'results.csv')
results_df.to_csv(all_results_path)
logger.info("DONE STEP 2")